In [2]:
import pandas as pd
import numpy as np

### 1. Import data

In [3]:
# maybe use np.where to reduce the data imported

Read

In [4]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [5]:
clinical = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [6]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [7]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv')

In [8]:
scripts = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

In [24]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

lines in samplescripts:  100000
number of subjects in samplescripts:  51104


Description of contents in datasets

In [9]:
code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

,coding,meaning
0,-1,redacted - potentially sensitive or identifying
1,-2,redacted - rare occupation
2,.....,Read thesaurus
3,.0...,See 0....
4,.01..,See 01...


In [10]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [11]:
ic10
# description of ICD20 codes

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
GPic10
#translates CTV3 codes (real_code) to ICD10 (icd10_code)

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 

# needs to be complete if we want to compare vaccinated vs. non-vaccinated, 
# but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,39278003,3743492,2021-03-22,320031002
1,35914327,3508920,2021-03-15,3591211000001105
2,49790799,4472075,2021-05-03,33405511000001102
3,31447155,3192040,2021-03-15,321745007
4,69964306,5879124,2021-01-19,320031002
...,...,...,...,...
99995,62996879,5393814,2021-01-27,320030001
99996,47834233,4333940,2021-01-03,38893411000001105
99997,33831040,3365447,2021-03-31,318052005
99998,12832421,1889162,2021-03-30,321152004


### 2. Correspond ICD10 to CTV3 and clean datasets

In [14]:
# new column in clinical dataset filled with ICD10 code

In [15]:
# drop all rows with ctv3 codes not corresponding to ICD10 (no diagnoses but measurements etc.)

In [16]:
# check if other rows have NaN

3. finding vaccinated people

In [39]:
# SNOMED code from https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

# dmd_code (SNOMED code) in 'scripts' = what medication patients got
# find dmd_code for COVID-vaccines on this website
# e.g. 39114911000001105 = AstraZeneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105


In [41]:
mask_PF = scripts['dmd_code']== 39114911000001105
scripts[mask_PF].head()

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105


In [43]:
# first, do not specify vaccines, but look only at vaccinated or not
# maybe specify later 

dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for code in dict_vac:
  mask_vac = scripts['dmd_code']== code
  print(dict_vac[code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 23
Pfizer : 7
Valneva : 0


# finding comorbidities

## IC10 codes

https://www.icd10data.com/ICD10CM/Codes
for easy lookup, divided into circulatory etc.

In [27]:
ic10

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


the GP_ic10 has the read code which is the CTV3. However, and when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc

In [29]:
# just to see if corresponding CTV3 to IC10 works
mask_vac_ic10 = GPic10['read_code'].isin(clinical[mask_vac]['code']) # broadcasting for vaccinated comorbidities(CTV3) in ICD10
GPic10[mask_vac_ic10]

# mask_vac: people that got the vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had: read_code: XE01qn


,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
76606,76606,XE0qn,R05X,D,C,C,0.0,0.0
76607,76607,XE0qn,F453,R,C,C,0.0,0.0
76608,76608,XE0qn,J111,R,C,C,0.0,0.0
76609,76609,XE0qn,J410,R,C,C,0.0,0.0
76610,76610,XE0qn,R042,R,C,C,0.0,0.0


## optional: investigating a list of specific diseases such as myocarditis

In [17]:
# mask for getting codes related to myocarditis

# change to searching through 'Description' of 'ic10' 

mask_myocarditis = code['meaning'].str.contains('myocarditis', case = False)
code[mask_myocarditis]

,coding,meaning
75314,A32y1,Diphtheritic myocarditis
75808,A7423,Coxsackie myocarditis
76092,A93y1,Syphilitic myocarditis
76517,AD03.,Toxoplasma myocarditis
86230,G012.,Acute rheumatic myocarditis
86282,G1y0.,Rheumatic myocarditis
86513,G52..,Acute myocarditis
86514,G520.,Acute myocarditis in diseases EC
86515,G5200,Acute aseptic myocarditis of the newborn
86516,G5201,Acute myocarditis - coxsackie


In [38]:
# code.loc[180816, 'meaning'] # super long meaning, may leave out

In [19]:
# code layout
clinical['selected specific diseases'] = np.nan # new column for filling with disease categories
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] # list of diseases, to be decided on

for disease in diseases_selec:
    # mask for specific disease
    mask = code['meaning'].str.contains(disease, case = False) 
    # subset clinical dataset using the codes corresponding to the disease
    sub_disease = clinical[clinical['code'].isin(code[mask]['coding'])] 
    
    for i in sub_disease.index:
        # fill the new column in the clinical dataset with disease category label
        clinical.loc[i,'selected specific diseases'] = disease 

In [27]:
clinical['selected specific diseases'].value_counts()

ischaemic heart disease    14
heart failure               1
Name: selected specific diseases, dtype: int64

# get comorbidities of vaccinated ones

In [25]:
id_vac = scripts[mask_PF]['eid']
# finding eids of vaccinated people

In [26]:
# finding events of the vaccinated people, but only events before vaccination found -> may need a more recent dataset
mask_vac = clinical['eid'].isin(id_vac)
clinical[mask_vac] 

,Unnamed: 0,eid,event_dt,code_type,code,value,selected specific diseases
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000,NaN
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998,NaN
36086,47865541,2608259,31/12/2019,0,242..,74.000000,NaN
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000,NaN
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000,NaN
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000,NaN
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000,NaN
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000,NaN
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000,NaN
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000,NaN


# code layout for getting categories of people w. different comorbidities

In [32]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

302       14A3.
303       14A4.
600       182..
602       1822.
610       1828.
          ...  
115198    Xafis
115199    Xafis
115200    Xafis
115204    Xafyq
115207    Xafyq
Name: read_code, Length: 3771, dtype: object

# From here on, there are still errors but the dataset will be changed (from code to ic10) anyways :)

In [33]:
# getting correspondign CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

TypeError: 'int' object is not subscriptable

In [34]:
clinical


,Unnamed: 0,eid,event_dt,code_type,code,value,selected specific diseases
0,101084316,4392480,31/12/2020,0,XaERu,11.00,NaN
1,9359934,1311969,31/12/2020,0,XaN3z,0.80,NaN
2,9360007,1311969,31/12/2020,0,44P6.,0.58,NaN
3,111269383,4731119,31/12/2020,0,XaIm8,0.00,NaN
4,9360002,1311969,31/12/2020,0,44P5.,1.30,NaN
...,...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00,NaN
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00,NaN
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00,NaN
49998,116371391,4900832,31/12/2019,0,X1004,0.00,NaN


In [35]:
# getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask
clinical[m_circ_sub]

NameError: name 'm_circ_CTV3' is not defined

# correspond to vaccinated or not

In [36]:
scripts[mask_vac]

C:\Users\moniz\AppData\Local\Temp/ipykernel_18668/870663266.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  scripts[mask_vac]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [37]:
# tbd: define timing (before or after vaccine?)

# no one w. pfizer had circulatory issue before vaccine on this limited dataset
scripts[mask_PF]['eid'].isin(clinical[m_circ_sub]['eid']).sum()

NameError: name 'm_circ_sub' is not defined